In [1]:
import numpy as np
import csv

from astropy.timeseries import BoxLeastSquares
from scipy.signal import savgol_filter
from astroquery.mast import Observations
import scipy.fftpack
from scipy import signal
from scipy.signal import savgol_filter
from scipy.stats import binned_statistic
import pandas as pd
from astroquery.mast import Catalogs
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pickle
from scipy.signal import savgol_filter
from astropy.coordinates import SkyCoord
from astroquery.mast import Tesscut
from astropy.io import fits
from astropy.wcs import WCS
from statistics import mode
from astropy.coordinates import SkyCoord
from astropy import coordinates as coord, units as u
import os
import glob
from astropy.modeling import models, fitting
from scipy.optimize import minimize, rosen, rosen_der
from astropy.modeling.models import custom_model
from scipy.optimize import curve_fit
from astropy.time import Time
from astropy.coordinates import SkyCoord
from datetime import datetime, timedelta
from scipy import stats

import importlib

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
plt.rcParams.update({'font.size': 20})

np.set_printoptions(precision=4)

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
path_ = '/Users/vkostov/Documents/GitHub/pandora-scheduler/src/pandorascheduler/data/'

In [89]:
target_list_name ='Pandora_Target_List_Top20_14May2024'
target_list_name = 'aux_list'
target_list =  target_list_name + '.csv'
target_list = pd.read_csv(path_ + target_list, sep=",")

# names=target_list['Star Name']

In [90]:
target_list

,Star Name,Star Simbad Name,RA,DEC,Flag
0,Gaia DR3 6523722346695237376,Gaia DR3 6523722346695237376,359.107751,-49.289017,0
1,Gaia DR3 1899442659524696832,Gaia DR3 1899442659524696832,330.873901,32.069177,0
2,Gaia DR3 4875300841790220672,Gaia DR3 4875300841790220672,76.717007,-31.776003,0
3,Gaia DR3 5341619081946297600,Gaia DR3 5341619081946297600,176.984979,-58.695391,0
4,Gaia DR3 6124986997511961344,Gaia DR3 6124986997511961344,180.325199,-53.559874,0
...,...,...,...,...,...
194,Gaia DR3 1540713861246325120,Gaia DR3 1540713861246325120,189.670556,44.289768,0
195,Gaia DR3 3583295892272722176,Gaia DR3 3583295892272722176,187.182113,-7.492819,0
196,Gaia DR3 2064855898933162624,Gaia DR3 2064855898933162624,308.840777,41.127994,0
197,Gaia DR3 6073017583988291840,Gaia DR3 6073017583988291840,186.729569,-55.209084,0


In [91]:
starts_stops = np.asarray([[61105.84097222239, 61105.85555555579], 
                 [61105.89930555551, 61105.92013888899], 
                 [61105.96388888871, 61105.98541666661], 
                 [61106.02916666679, 61106.059027778], 
                 [61106.08749999991, 61106.133333333295], 
                 [61106.14444444421, 61106.83958333312]])
starts, stops = starts_stops[:,0], starts_stops[:,1]

In [97]:
ii = 0

for ii in range(len(target_list)):
    for s in range(len(starts)):

        star_name = target_list["Star Name"][ii]
        vis = pd.read_csv(path_ + f'/aux_targets/{star_name}/Visibility for {star_name}.csv')

        vis_ = vis['Time(MJD_UTC)']

        ast = set(vis.index[vis_ >= starts[s]])
        bsp = set(vis.index[vis_ <= stops[s]])
        win = list(ast.intersection(bsp))
        
        if len(vis['Visible'][win]) > 0:
            vis_ratio = len(vis['Visible'][win][vis['Visible'][win] == 1])/len(vis['Visible'][win])

        if np.all(vis['Visible'][win] == 1) or np.any(vis['Visible'][win] == 1):
            print(star_name, "%0.2f" % starts[s], "%0.2f" % stops[s], "%0.2f" %  (24.*(stops[s] - starts[s])), \
                  np.all(vis['Visible'][win] == 1), np.any(vis['Visible'][win] == 1), "%0.2f" % vis_ratio)
    print()
#     plt.plot(vis['Time(MJD_UTC)'][vis['Visible'] == 1.], vis['Visible'][vis['Visible'] == 1.], '.')




Gaia DR3 5341619081946297600 61105.84 61105.86 0.35 True True 1.00
Gaia DR3 5341619081946297600 61105.90 61105.92 0.50 False True 0.83
Gaia DR3 5341619081946297600 61105.96 61105.99 0.52 False True 0.78
Gaia DR3 5341619081946297600 61106.03 61106.06 0.72 False True 0.86
Gaia DR3 5341619081946297600 61106.09 61106.13 1.10 False True 0.76
Gaia DR3 5341619081946297600 61106.14 61106.84 16.68 False True 0.69

Gaia DR3 6124986997511961344 61105.84 61105.86 0.35 True True 1.00
Gaia DR3 6124986997511961344 61105.90 61105.92 0.50 False True 0.83
Gaia DR3 6124986997511961344 61105.96 61105.99 0.52 False True 0.78
Gaia DR3 6124986997511961344 61106.03 61106.06 0.72 False True 0.86
Gaia DR3 6124986997511961344 61106.09 61106.13 1.10 False True 0.76
Gaia DR3 6124986997511961344 61106.14 61106.84 16.68 False True 0.71

Gaia DR3 5795448244374261120 61105.84 61105.86 0.35 False True 0.91
Gaia DR3 5795448244374261120 61105.90 61105.92 0.50 False True 0.67
Gaia DR3 5795448244374261120 61105.96 61105

Gaia DR3 5348895443728025472 61105.96 61105.99 0.52 False True 0.88
Gaia DR3 5348895443728025472 61106.03 61106.06 0.72 False True 0.91
Gaia DR3 5348895443728025472 61106.09 61106.13 1.10 False True 0.80
Gaia DR3 5348895443728025472 61106.14 61106.84 16.68 False True 0.72


Gaia DR3 5368986304315428736 61105.84 61105.86 0.35 True True 1.00
Gaia DR3 5368986304315428736 61105.90 61105.92 0.50 False True 0.83
Gaia DR3 5368986304315428736 61105.96 61105.99 0.52 False True 0.81
Gaia DR3 5368986304315428736 61106.03 61106.06 0.72 False True 0.86
Gaia DR3 5368986304315428736 61106.09 61106.13 1.10 False True 0.77
Gaia DR3 5368986304315428736 61106.14 61106.84 16.68 False True 0.71


Gaia DR3 5877742532239591296 61105.84 61105.86 0.35 False True 0.86
Gaia DR3 5877742532239591296 61105.90 61105.92 0.50 False True 0.60
Gaia DR3 5877742532239591296 61105.96 61105.99 0.52 False True 0.59
Gaia DR3 5877742532239591296 61106.03 61106.06 0.72 False True 0.70
Gaia DR3 5877742532239591296 61106.09 61106

Gaia DR3 5444028866255657856 61106.14 61106.84 16.68 False True 0.84

Gaia DR3 5704189981858788736 61105.84 61105.86 0.35 True True 1.00
Gaia DR3 5704189981858788736 61105.90 61105.92 0.50 True True 1.00
Gaia DR3 5704189981858788736 61105.96 61105.99 0.52 True True 1.00
Gaia DR3 5704189981858788736 61106.03 61106.06 0.72 True True 1.00
Gaia DR3 5704189981858788736 61106.09 61106.13 1.10 False True 0.98
Gaia DR3 5704189981858788736 61106.14 61106.84 16.68 False True 0.75




Gaia DR3 5391695480028231040 61105.84 61105.86 0.35 True True 1.00
Gaia DR3 5391695480028231040 61105.90 61105.92 0.50 True True 1.00
Gaia DR3 5391695480028231040 61105.96 61105.99 0.52 True True 1.00
Gaia DR3 5391695480028231040 61106.03 61106.06 0.72 True True 1.00
Gaia DR3 5391695480028231040 61106.09 61106.13 1.10 False True 0.88
Gaia DR3 5391695480028231040 61106.14 61106.84 16.68 False True 0.78


Gaia DR3 5623061932292559616 61105.84 61105.86 0.35 True True 1.00
Gaia DR3 5623061932292559616 61105.90 61105.92 

Gaia DR3 1394100999363958656 61105.96 61105.99 0.52 False True 0.38
Gaia DR3 1394100999363958656 61106.03 61106.06 0.72 False True 0.27
Gaia DR3 1394100999363958656 61106.09 61106.13 1.10 False True 0.52
Gaia DR3 1394100999363958656 61106.14 61106.84 16.68 False True 0.65




Gaia DR3 1360962985919129344 61106.14 61106.84 16.68 False True 0.21

Gaia DR3 5872456836203586560 61105.84 61105.86 0.35 False True 0.86
Gaia DR3 5872456836203586560 61105.90 61105.92 0.50 False True 0.63
Gaia DR3 5872456836203586560 61105.96 61105.99 0.52 False True 0.59
Gaia DR3 5872456836203586560 61106.03 61106.06 0.72 False True 0.73
Gaia DR3 5872456836203586560 61106.09 61106.13 1.10 False True 0.67
Gaia DR3 5872456836203586560 61106.14 61106.84 16.68 False True 0.25


Gaia DR3 5865637287140976640 61105.84 61105.86 0.35 False True 0.91
Gaia DR3 5865637287140976640 61105.90 61105.92 0.50 False True 0.67
Gaia DR3 5865637287140976640 61105.96 61105.99 0.52 False True 0.62
Gaia DR3 5865637287140976640 61106.03 

In [ ]:
Time(2400000.5 + vis['Time(MJD_UTC)'], format = 'jd').iso

time_iso_visible = Time(2400000.5 + vis['Time(MJD_UTC)'], format = 'jd').iso#[vis['Visible'] == 1.]
time_visible = vis['Time(MJD_UTC)']
visible_ = vis['Visible']#[vis['Visible'] == 1.]

In [18]:
times = ['2026-3-7T3:27:59']#, '2010-01-01T00:00:00']

t = Time(times, format='isot', scale='utc')
t.mjd

array([61106.1444])

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w
#lc_SC = lk.search_lightcurvefile('TIC ' + str(TIC), sector = 33).download()
#tt, flux_SAP, flux_PDC = lc_SC.time, lc_SC.SAP_FLUX.flux, lc_SC.PDCSAP_FLUX.flux
#SAP_10 = moving_average(flux_SAP/np.nanmean(flux_SAP), 100)

In [ ]:
np.asarray(time_iso_visible)

In [ ]:
nbins = int(len(visible_)/60)
bin_means_tmp, bin_edges_tmp, binnumber = stats.binned_statistic(time_visible, visible_, statistic='mean', bins=nbins)
bin_width_tmp = (bin_edges_tmp[1] - bin_edges_tmp[0])
bin_centers_tmp = bin_edges_tmp[1:] - bin_width_tmp/2

In [ ]:
fig = plt.figure(figsize=(20,5))
plt.rcParams.update({'font.size': 26}) 
plt.plot(bin_centers_tmp, 60*bin_means_tmp, 'k.-')
plt.xlim(60928, 60935)


In [ ]:
plt.plot(vis['Time(MJD_UTC)'][vis['Visible'] == 1.], vis['Visible'][vis['Visible'] == 1.], '.')
# plt.plot(time_iso, vis['Visible'][vis['Visible'] == 1.], '.')

In [ ]:
i = 0
path_ = '/Users/vkostov/Documents/GitHub/pandora-scheduler/src/pandorascheduler/data/'
planet_name = target_list["Planet Name"][i]
star_name = target_list["Star Name"][i]
planet_data = pd.read_csv(path_ + f'/targets/{star_name}/{planet_name}/Visibility for {planet_name}.csv')

In [ ]:
planet_data

In [ ]:
fig = plt.figure(figsize=(20,11))
plt.rcParams.update({'font.size': 26})  